# Reading data

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle

In [2]:
PATH = os.path.join("../data", "train_scaled_2.0.csv")
data = pd.read_csv(PATH)
data = shuffle(data)
data = data.dropna()

In [3]:
# data["gender"] = data["gender"].map({"male": 1, "female": 0})

In [4]:
# data["age_60_and_above"] = data["age_60_and_above"].map({"Yes": 1, "No": 0})

In [5]:
# data["test_indication"] = data["test_indication"].map({"Contact with confirmed": 2, "Abroad": 1, "Other": 0})

In [6]:
data

,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age,gender,test_indication
68371,1.0,0.0,1.0,0.0,1.0,0.0,62.0,1.0,0.0
95675,2.0,1.0,0.0,1.0,0.0,0.0,62.0,0.0,0.0
98713,0.0,1.0,0.0,0.0,0.0,0.0,70.0,1.0,0.0
14637,0.0,0.0,1.0,0.0,0.0,0.0,41.0,0.0,0.0
1782,0.0,1.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
46505,2.0,2.0,0.0,0.0,1.0,0.0,64.0,0.0,1.0
48366,0.0,0.0,1.0,0.0,0.0,0.0,21.0,0.0,0.0
51768,1.0,1.0,0.0,0.0,0.0,0.0,55.0,0.0,0.0
22171,0.0,0.0,0.0,0.0,0.0,0.0,64.0,1.0,0.0


In [7]:
data["corona_result"].value_counts(dropna = False)

0.0    98586
1.0     8956
Name: corona_result, dtype: int64

In [8]:
y = data["corona_result"]
X = data.drop(["corona_result"], axis = 1)

In [9]:
X["gender"].unique()

array([1., 0.])

In [10]:
y.unique()

array([0., 1.])

In [11]:
X.shape, y.shape, data.shape, data["corona_result"].shape

((107542, 8), (107542,), (107542, 9), (107542,))

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.22, random_state = 11)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((83882, 8), (23660, 8), (83882,), (23660,))

# Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
# Cs = [0.0001, 0.00001]
logit = LogisticRegression(class_weight = "balanced", \
                             n_jobs = -1, verbose = 5, random_state = 11)

In [17]:
logit.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.2s finished


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=11, solver='lbfgs', tol=0.0001, verbose=5,
                   warm_start=False)

In [19]:
# logit.C_

In [20]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
accuracy_score(y_test, logit.predict(X_test)), recall_score(y_test, logit.predict(X_test)), \
            precision_score(y_test, logit.predict(X_test))

(0.8983516483516484, 0.8154311649016641, 0.4422866520787746)

In [21]:
y_test.value_counts()

0.0    21677
1.0     1983
Name: corona_result, dtype: int64

### not the best result

In [165]:
result = logit.predict(X_test)
for i in range(len(result)):
    print(i, int(result[i]), int(y_test.iloc[i]))

0 1 0
1 0 0
2 0 0
3 0 0
4 0 0
5 1 0
6 0 0
7 0 0
8 0 0
9 0 0
10 1 1
11 0 0
12 0 0
13 1 1
14 0 1
15 1 0
16 0 0
17 0 0
18 0 0
19 1 0
20 0 0
21 0 0
22 0 0
23 0 0
24 0 0
25 0 0
26 1 1
27 0 0
28 0 0
29 0 0
30 0 0
31 1 1
32 1 1
33 1 0
34 1 1
35 0 0
36 0 0
37 0 0
38 0 0
39 0 0
40 0 0
41 1 1
42 0 0
43 0 0
44 1 1
45 0 0
46 0 0
47 0 1
48 0 0
49 0 0
50 0 0
51 0 0
52 0 0
53 0 0
54 1 1
55 0 0
56 0 0
57 0 0
58 0 0
59 0 0
60 0 0
61 0 0
62 0 0
63 0 0
64 0 0
65 0 0
66 0 0
67 1 1
68 0 0
69 0 0
70 1 0
71 0 0
72 0 0
73 0 0
74 0 0
75 1 1
76 0 0
77 1 0
78 0 0
79 0 0
80 0 0
81 0 0
82 0 0
83 0 0
84 1 1
85 0 0
86 0 0
87 0 0
88 0 0
89 0 0
90 0 0
91 0 0
92 0 0
93 0 0
94 1 0
95 0 0
96 1 1
97 1 0
98 0 0
99 0 0
100 0 0
101 0 0
102 1 0
103 0 0
104 1 0
105 1 0
106 0 0
107 0 0
108 0 0
109 0 1
110 1 0
111 0 0
112 0 0
113 0 0
114 1 0
115 0 0
116 0 1
117 0 0
118 0 0
119 1 0
120 0 0
121 0 0
122 1 0
123 0 0
124 0 0
125 0 0
126 0 0
127 1 1
128 1 0
129 0 0
130 0 0
131 0 0
132 1 0
133 1 1
134 0 0
135 0 0
136 0 0
137 0 0
138 0 

1061 0 0
1062 0 0
1063 0 0
1064 0 0
1065 1 0
1066 1 0
1067 0 0
1068 0 0
1069 1 0
1070 0 0
1071 0 0
1072 1 1
1073 0 0
1074 0 0
1075 0 0
1076 0 0
1077 0 0
1078 0 0
1079 0 0
1080 1 0
1081 0 0
1082 0 0
1083 0 0
1084 1 1
1085 0 0
1086 0 0
1087 0 0
1088 0 0
1089 1 0
1090 0 0
1091 0 0
1092 0 0
1093 1 0
1094 0 0
1095 0 0
1096 0 0
1097 0 0
1098 0 0
1099 0 0
1100 1 1
1101 0 0
1102 0 0
1103 0 0
1104 0 0
1105 1 0
1106 1 0
1107 1 0
1108 0 1
1109 0 0
1110 0 0
1111 0 0
1112 0 0
1113 0 0
1114 0 0
1115 0 0
1116 0 0
1117 0 0
1118 0 0
1119 1 1
1120 1 0
1121 0 0
1122 0 0
1123 1 1
1124 0 0
1125 0 0
1126 0 0
1127 1 0
1128 0 0
1129 0 0
1130 0 0
1131 0 0
1132 0 0
1133 0 0
1134 0 0
1135 0 0
1136 0 0
1137 0 0
1138 0 0
1139 0 0
1140 1 0
1141 0 0
1142 0 0
1143 0 0
1144 0 0
1145 0 0
1146 0 0
1147 0 0
1148 0 0
1149 0 0
1150 0 0
1151 0 0
1152 0 0
1153 0 0
1154 0 0
1155 0 0
1156 0 0
1157 0 0
1158 0 0
1159 0 0
1160 0 0
1161 0 0
1162 0 0
1163 0 0
1164 0 0
1165 0 0
1166 0 0
1167 0 0
1168 0 0
1169 0 0
1170 1 0
1171 0 0
1

1983 0 0
1984 1 1
1985 0 0
1986 0 0
1987 0 0
1988 0 0
1989 0 0
1990 0 0
1991 0 0
1992 0 0
1993 0 0
1994 0 0
1995 0 0
1996 0 0
1997 0 0
1998 0 0
1999 0 0
2000 0 0
2001 0 0
2002 0 0
2003 0 0
2004 0 0
2005 1 1
2006 0 0
2007 0 0
2008 0 0
2009 0 0
2010 0 0
2011 1 0
2012 0 0
2013 0 0
2014 0 0
2015 0 0
2016 0 0
2017 1 0
2018 0 0
2019 1 0
2020 0 0
2021 0 0
2022 0 0
2023 1 1
2024 0 0
2025 0 0
2026 0 0
2027 0 0
2028 0 0
2029 0 0
2030 0 0
2031 0 0
2032 0 0
2033 0 0
2034 1 0
2035 0 0
2036 0 0
2037 0 0
2038 0 0
2039 0 0
2040 0 0
2041 0 0
2042 0 0
2043 0 0
2044 1 1
2045 0 0
2046 0 0
2047 0 0
2048 0 0
2049 1 1
2050 0 0
2051 0 0
2052 0 0
2053 0 0
2054 0 0
2055 0 0
2056 0 0
2057 0 0
2058 0 0
2059 0 0
2060 0 0
2061 1 0
2062 0 0
2063 0 0
2064 0 0
2065 0 0
2066 0 0
2067 0 0
2068 0 0
2069 0 0
2070 0 0
2071 0 0
2072 0 0
2073 1 1
2074 0 0
2075 0 0
2076 0 0
2077 1 0
2078 0 0
2079 1 1
2080 0 0
2081 0 0
2082 0 0
2083 0 0
2084 0 0
2085 0 0
2086 0 0
2087 0 0
2088 0 0
2089 0 0
2090 0 0
2091 0 0
2092 0 0
2093 0 0
2

3203 0 0
3204 0 0
3205 1 1
3206 1 1
3207 0 0
3208 0 0
3209 0 0
3210 0 0
3211 0 0
3212 1 1
3213 1 1
3214 0 0
3215 0 0
3216 0 0
3217 0 0
3218 0 0
3219 0 0
3220 0 0
3221 0 0
3222 0 0
3223 0 0
3224 0 0
3225 1 1
3226 1 0
3227 0 0
3228 0 0
3229 0 0
3230 0 1
3231 1 1
3232 0 0
3233 0 0
3234 0 0
3235 0 0
3236 0 0
3237 0 0
3238 0 0
3239 0 0
3240 1 1
3241 1 1
3242 0 0
3243 0 0
3244 0 1
3245 0 0
3246 0 0
3247 0 0
3248 0 0
3249 0 0
3250 0 0
3251 0 0
3252 0 1
3253 0 0
3254 0 0
3255 0 0
3256 1 0
3257 1 1
3258 0 0
3259 0 0
3260 0 0
3261 1 0
3262 0 0
3263 0 0
3264 0 0
3265 0 0
3266 0 0
3267 0 0
3268 0 0
3269 0 0
3270 0 0
3271 0 0
3272 1 1
3273 0 0
3274 0 0
3275 0 0
3276 0 0
3277 0 0
3278 1 1
3279 0 0
3280 0 0
3281 0 0
3282 0 0
3283 0 0
3284 0 0
3285 0 0
3286 1 0
3287 0 0
3288 0 0
3289 0 0
3290 0 0
3291 0 0
3292 0 0
3293 1 1
3294 0 0
3295 1 0
3296 0 0
3297 0 0
3298 0 0
3299 1 0
3300 0 0
3301 0 0
3302 1 0
3303 0 0
3304 0 0
3305 0 0
3306 0 0
3307 0 0
3308 0 0
3309 0 0
3310 1 0
3311 0 0
3312 0 0
3313 1 0
3

4531 0 0
4532 0 0
4533 0 0
4534 1 1
4535 0 0
4536 0 0
4537 0 0
4538 0 0
4539 1 0
4540 0 0
4541 0 0
4542 0 0
4543 0 0
4544 0 0
4545 0 0
4546 0 0
4547 0 0
4548 0 0
4549 1 0
4550 0 0
4551 1 0
4552 0 0
4553 0 0
4554 1 0
4555 0 0
4556 0 0
4557 0 0
4558 0 0
4559 0 0
4560 0 0
4561 0 0
4562 0 0
4563 0 0
4564 0 0
4565 0 0
4566 0 0
4567 0 0
4568 0 0
4569 0 0
4570 0 0
4571 0 0
4572 0 0
4573 0 0
4574 0 0
4575 1 1
4576 0 0
4577 0 0
4578 0 0
4579 0 0
4580 0 0
4581 0 0
4582 0 0
4583 1 0
4584 0 0
4585 0 0
4586 0 0
4587 0 0
4588 0 0
4589 0 0
4590 0 0
4591 0 0
4592 0 0
4593 0 0
4594 0 0
4595 0 0
4596 0 0
4597 0 0
4598 0 0
4599 0 0
4600 1 1
4601 0 0
4602 0 0
4603 0 0
4604 0 0
4605 0 0
4606 0 0
4607 1 1
4608 1 0
4609 1 0
4610 0 0
4611 0 0
4612 0 0
4613 0 0
4614 0 0
4615 1 1
4616 0 0
4617 0 0
4618 0 0
4619 0 0
4620 0 0
4621 1 0
4622 0 0
4623 1 1
4624 0 0
4625 0 0
4626 0 0
4627 0 0
4628 0 0
4629 0 0
4630 0 0
4631 1 1
4632 1 1
4633 0 0
4634 1 0
4635 0 0
4636 0 0
4637 0 0
4638 1 0
4639 0 0
4640 0 0
4641 0 0
4

...

18484 0 1
18485 0 0
18486 0 0
18487 0 0
18488 1 1
18489 0 0
18490 0 0
18491 0 0
18492 0 0
18493 1 1
18494 0 0
18495 1 0
18496 0 0
18497 0 0
18498 0 0
18499 0 0
18500 0 0
18501 0 0
18502 0 0
18503 0 0
18504 0 0
18505 0 0
18506 0 0
18507 1 0
18508 0 0
18509 0 1
18510 0 0
18511 0 0
18512 0 0
18513 0 0
18514 0 0
18515 0 0
18516 0 0
18517 0 0
18518 0 0
18519 0 0
18520 0 0
18521 0 0
18522 0 0
18523 0 0
18524 0 0
18525 0 0
18526 0 0
18527 1 1
18528 1 1
18529 1 0
18530 0 0
18531 0 0
18532 0 0
18533 0 0
18534 0 0
18535 0 0
18536 0 0
18537 0 0
18538 0 0
18539 0 0
18540 0 0
18541 0 0
18542 0 0
18543 0 0
18544 0 0
18545 0 0
18546 0 0
18547 0 0
18548 0 0
18549 0 0
18550 0 0
18551 0 0
18552 0 0
18553 0 0
18554 0 1
18555 0 0
18556 0 0
18557 0 0
18558 0 0
18559 0 0
18560 0 0
18561 0 0
18562 0 0
18563 0 0
18564 1 0
18565 0 0
18566 0 0
18567 0 0
18568 1 1
18569 0 0
18570 1 0
18571 0 0
18572 0 0
18573 1 0
18574 0 0
18575 0 0
18576 0 0
18577 0 0
18578 0 0
18579 0 0
18580 0 0
18581 1 1
18582 0 0
18583 0 0


19811 0 0
19812 0 0
19813 0 0
19814 0 0
19815 1 0
19816 1 0
19817 0 0
19818 0 0
19819 1 0
19820 0 0
19821 0 0
19822 0 0
19823 0 0
19824 0 0
19825 0 0
19826 0 0
19827 0 0
19828 0 0
19829 0 0
19830 0 0
19831 0 0
19832 1 0
19833 0 0
19834 0 0
19835 0 0
19836 1 0
19837 0 0
19838 1 0
19839 0 0
19840 0 0
19841 0 0
19842 1 0
19843 0 0
19844 0 0
19845 0 0
19846 1 1
19847 0 0
19848 0 0
19849 0 0
19850 0 0
19851 0 0
19852 0 0
19853 0 0
19854 0 0
19855 0 0
19856 0 0
19857 1 0
19858 0 0
19859 0 0
19860 0 0
19861 1 0
19862 1 1
19863 0 0
19864 1 1
19865 0 0
19866 0 0
19867 0 0
19868 1 1
19869 0 0
19870 0 0
19871 0 0
19872 0 0
19873 0 0
19874 0 0
19875 1 1
19876 1 0
19877 0 0
19878 0 0
19879 0 0
19880 1 0
19881 0 0
19882 0 0
19883 0 0
19884 0 0
19885 0 0
19886 0 0
19887 0 0
19888 0 0
19889 1 0
19890 0 0
19891 0 0
19892 1 0
19893 0 0
19894 0 0
19895 0 0
19896 1 0
19897 1 0
19898 0 0
19899 0 0
19900 0 0
19901 1 0
19902 1 1
19903 1 0
19904 1 1
19905 0 0
19906 0 0
19907 0 0
19908 1 1
19909 0 0
19910 0 0


20785 0 0
20786 0 0
20787 0 0
20788 0 0
20789 0 0
20790 0 0
20791 1 0
20792 0 0
20793 0 0
20794 0 0
20795 1 0
20796 0 0
20797 0 0
20798 1 0
20799 0 0
20800 1 1
20801 0 0
20802 0 0
20803 1 1
20804 1 1
20805 0 0
20806 0 0
20807 0 1
20808 0 0
20809 0 0
20810 1 0
20811 0 0
20812 0 0
20813 0 0
20814 0 0
20815 0 0
20816 0 0
20817 0 0
20818 0 0
20819 0 0
20820 0 0
20821 0 0
20822 0 0
20823 1 1
20824 0 0
20825 1 0
20826 0 0
20827 0 0
20828 0 0
20829 0 0
20830 1 1
20831 0 0
20832 0 0
20833 0 0
20834 0 0
20835 0 1
20836 0 0
20837 0 0
20838 0 0
20839 0 0
20840 0 0
20841 0 0
20842 1 0
20843 0 0
20844 0 0
20845 0 0
20846 0 0
20847 0 0
20848 0 0
20849 0 0
20850 0 0
20851 0 0
20852 1 0
20853 0 0
20854 0 0
20855 0 0
20856 0 0
20857 0 0
20858 0 0
20859 1 0
20860 1 0
20861 1 0
20862 0 0
20863 0 0
20864 0 0
20865 0 0
20866 0 0
20867 0 0
20868 0 0
20869 0 0
20870 0 0
20871 0 0
20872 1 0
20873 1 0
20874 1 0
20875 1 0
20876 0 0
20877 0 0
20878 0 0
20879 0 0
20880 0 0
20881 0 0
20882 0 0
20883 1 1
20884 1 1


21782 0 0
21783 0 0
21784 0 0
21785 0 0
21786 0 0
21787 0 0
21788 0 0
21789 0 0
21790 0 0
21791 0 0
21792 0 0
21793 1 0
21794 0 0
21795 0 0
21796 1 0
21797 0 0
21798 0 0
21799 1 1
21800 0 0
21801 0 0
21802 0 0
21803 0 0
21804 0 0
21805 0 0
21806 0 0
21807 0 0
21808 0 0
21809 0 0
21810 0 0
21811 0 0
21812 0 0
21813 1 1
21814 0 0
21815 1 1
21816 0 0
21817 0 0
21818 0 0
21819 1 0
21820 0 0
21821 1 1
21822 0 0
21823 0 0
21824 0 0
21825 0 0
21826 0 0
21827 0 0
21828 0 0
21829 1 0
21830 0 0
21831 0 0
21832 0 0
21833 0 0
21834 0 0
21835 0 0
21836 1 0
21837 0 0
21838 0 0
21839 0 0
21840 1 0
21841 0 0
21842 0 0
21843 0 1
21844 1 1
21845 0 0
21846 1 0
21847 0 0
21848 1 0
21849 0 0
21850 0 0
21851 0 0
21852 0 0
21853 0 0
21854 0 0
21855 0 0
21856 0 0
21857 0 0
21858 0 0
21859 0 0
21860 0 0
21861 0 0
21862 0 0
21863 0 0
21864 1 0
21865 1 0
21866 0 0
21867 0 0
21868 0 0
21869 0 0
21870 0 0
21871 0 0
21872 1 1
21873 0 0
21874 0 0
21875 0 0
21876 1 0
21877 1 0
21878 1 0
21879 0 0
21880 1 1
21881 0 0


22682 0 0
22683 0 0
22684 0 0
22685 0 0
22686 0 0
22687 0 0
22688 0 0
22689 0 0
22690 1 0
22691 0 0
22692 0 0
22693 0 0
22694 0 0
22695 0 0
22696 0 0
22697 1 0
22698 0 0
22699 0 0
22700 0 0
22701 0 0
22702 0 0
22703 0 0
22704 0 0
22705 0 0
22706 1 0
22707 0 0
22708 0 0
22709 0 0
22710 0 0
22711 1 0
22712 0 0
22713 0 0
22714 1 1
22715 1 1
22716 0 0
22717 0 0
22718 0 0
22719 0 0
22720 0 0
22721 0 0
22722 0 0
22723 0 0
22724 0 0
22725 0 0
22726 0 0
22727 0 0
22728 0 0
22729 0 0
22730 0 0
22731 0 0
22732 0 0
22733 0 0
22734 0 0
22735 1 1
22736 0 0
22737 0 0
22738 0 0
22739 0 0
22740 0 0
22741 0 0
22742 1 0
22743 0 0
22744 1 0
22745 0 0
22746 0 0
22747 0 0
22748 0 0
22749 0 0
22750 0 0
22751 0 0
22752 0 0
22753 0 0
22754 0 0
22755 0 0
22756 0 0
22757 0 0
22758 0 0
22759 0 0
22760 0 0
22761 0 0
22762 0 0
22763 0 0
22764 0 0
22765 0 0
22766 0 0
22767 0 0
22768 0 0
22769 0 0
22770 0 0
22771 0 0
22772 0 0
22773 1 1
22774 0 0
22775 0 0
22776 0 0
22777 1 0
22778 0 0
22779 0 0
22780 0 0
22781 0 0


# Random forest

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(n_estimators = 201, max_depth = 16, \
                            oob_score = True, n_jobs = -1, class_weight = "balanced", random_state = 17)

# params = {
#     "max_depth": np.arange(15, 19)
# }

# # depth = 11
# # depth = 18

# rf = GridSearchCV(rf, params, n_jobs = -1, verbose = 30)

In [46]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=16, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=201,
                       n_jobs=-1, oob_score=True, random_state=17, verbose=0,
                       warm_start=False)

In [47]:
# rf.best_params_

In [48]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
accuracy = accuracy_score(y_test, rf.predict(X_test))
recall = recall_score(y_test, rf.predict(X_test))
precision = precision_score(y_test, rf.predict(X_test))

print("Общая точность: {}".format(accuracy))
print("Полнота (определение больного человека): {}".format(recall))
print("Точность положительного диагноза: {}".format(precision))

Общая точность: 0.9686390532544379
Полнота (определение больного человека): 0.7760968229954615
Точность положительного диагноза: 0.8377789874795862


In [63]:
total = 0
middle = 0
for elem in rf.predict_proba(X_test):
    if 0.25 <= elem[0] <= 0.75:
        middle += 1
        print(elem)
    total += 1

[0.65201989 0.34798011]
[0.58440061 0.41559939]
[0.66675168 0.33324832]
[0.47625663 0.52374337]
[0.31865734 0.68134266]
[0.59785383 0.40214617]
[0.69031012 0.30968988]
[0.4481952 0.5518048]
[0.67909575 0.32090425]
[0.74814055 0.25185945]
[0.72392849 0.27607151]
[0.49136343 0.50863657]
[0.71794649 0.28205351]
[0.44069899 0.55930101]
[0.55928497 0.44071503]
[0.63902684 0.36097316]
[0.71562601 0.28437399]
[0.73709625 0.26290375]
[0.6495219 0.3504781]
[0.69366151 0.30633849]
[0.74124273 0.25875727]
[0.28825716 0.71174284]
[0.26953112 0.73046888]
[0.60833838 0.39166162]
[0.74466385 0.25533615]
[0.68571064 0.31428936]
[0.5583787 0.4416213]
[0.65924939 0.34075061]
[0.52639463 0.47360537]
[0.35831406 0.64168594]
[0.40406599 0.59593401]
[0.62591959 0.37408041]
[0.4724294 0.5275706]
[0.52880847 0.47119153]
[0.27958356 0.72041644]
[0.74124273 0.25875727]
[0.74593518 0.25406482]
[0.72729718 0.27270282]
[0.39091491 0.60908509]
[0.65617273 0.34382727]
[0.50005554 0.49994446]
[0.62827959 0.37172041]


[0.55509281 0.44490719]
[0.68630253 0.31369747]
[0.30886877 0.69113123]
[0.73128174 0.26871826]
[0.64009037 0.35990963]
[0.73652577 0.26347423]
[0.679034 0.320966]
[0.63798068 0.36201932]
[0.61203472 0.38796528]
[0.60910652 0.39089348]
[0.62893879 0.37106121]
[0.60640203 0.39359797]
[0.69187474 0.30812526]
[0.73332302 0.26667698]
[0.65450883 0.34549117]
[0.51255831 0.48744169]
[0.69751947 0.30248053]
[0.59193315 0.40806685]
[0.72397397 0.27602603]
[0.72686349 0.27313651]
[0.69031012 0.30968988]
[0.47565734 0.52434266]
[0.72790556 0.27209444]
[0.51846932 0.48153068]
[0.73877425 0.26122575]
[0.74635067 0.25364933]
[0.73531389 0.26468611]
[0.67753673 0.32246327]
[0.56488805 0.43511195]
[0.74031903 0.25968097]
[0.67087624 0.32912376]
[0.70327403 0.29672597]
[0.49949503 0.50050497]
[0.72771281 0.27228719]
[0.3345496 0.6654504]
[0.73996271 0.26003729]
[0.74477057 0.25522943]
[0.68532337 0.31467663]
[0.4499887 0.5500113]
[0.46682685 0.53317315]
[0.5388888 0.4611112]
[0.71299498 0.28700502]
[0

In [64]:
print(total, middle)

23660 1452


In [99]:
for elem in rf.predict_proba(X_test):
    print(elem)

[0.86397663 0.13602337]
[0.86317864 0.13682136]
[0.7723277 0.2276723]
[0.82775886 0.17224114]
[0.89621567 0.10378433]
[0.96196647 0.03803353]
[0.86043568 0.13956432]
[0.86759952 0.13240048]
[0.00497512 0.99502488]
[0.85237687 0.14762313]
[0.91086396 0.08913604]
[0. 1.]
[0.79341862 0.20658138]
[0.84364087 0.15635913]
[0.82777345 0.17222655]
[0.87481067 0.12518933]
[0.86947536 0.13052464]
[0.09338285 0.90661715]
[0.85684374 0.14315626]
[0.88634415 0.11365585]
[0.89935893 0.10064107]
[0.86212757 0.13787243]
[0.90176759 0.09823241]
[0.86917186 0.13082814]
[0.89836686 0.10163314]
[0.91154184 0.08845816]
[0.90248709 0.09751291]
[0.91421162 0.08578838]
[0.87859653 0.12140347]
[0.8873231 0.1126769]
[0.81713134 0.18286866]
[0.88715493 0.11284507]
[0.86412448 0.13587552]
[0.87064363 0.12935637]
[0.89233308 0.10766692]
[0.87587075 0.12412925]
[0.85362814 0.14637186]
[0.88632693 0.11367307]
[0.86660532 0.13339468]
[0.83314521 0.16685479]
[0.86263423 0.13736577]
[0.9070418 0.0929582]
[0.85940185 0.

[0.89244487 0.10755513]
[0.64390053 0.35609947]
[0.89562119 0.10437881]
[0.12522262 0.87477738]
[0.77152663 0.22847337]
[0.86716819 0.13283181]
[0.8703526 0.1296474]
[0.89026958 0.10973042]
[0.8738579 0.1261421]
[0.86919793 0.13080207]
[0.8435838 0.1564162]
[0.85813958 0.14186042]
[0.8739996 0.1260004]
[0.76865961 0.23134039]
[0.90519193 0.09480807]
[0.89112343 0.10887657]
[0.83102111 0.16897889]
[0.85341618 0.14658382]
[0.8645017 0.1354983]
[0.01344062 0.98655938]
[0.85975625 0.14024375]
[0.85081237 0.14918763]
[0.89525526 0.10474474]
[0.89356121 0.10643879]
[0.83019852 0.16980148]
[0.9050038 0.0949962]
[0.90941864 0.09058136]
[0.84534846 0.15465154]
[0.88020416 0.11979584]
[0.78669741 0.21330259]
[0.88660181 0.11339819]
[0.86011713 0.13988287]
[0.85284004 0.14715996]
[0.85303792 0.14696208]
[0.86385146 0.13614854]
[0.84832379 0.15167621]
[0.90280374 0.09719626]
[0.83010263 0.16989737]
[0.87865444 0.12134556]
[0.88534518 0.11465482]
[0.89355199 0.10644801]
[0.83824526 0.16175474]
[0.8

[0.89840449 0.10159551]
[0.7398746 0.2601254]
[0.84419275 0.15580725]
[0.89451283 0.10548717]
[0.89941362 0.10058638]
[0.89337897 0.10662103]
[0.86938034 0.13061966]
[0.75827142 0.24172858]
[0.88473863 0.11526137]
[0.89849166 0.10150834]
[0.86237242 0.13762758]
[0.86597518 0.13402482]
[0.7181864 0.2818136]
[0.86707308 0.13292692]
[0.88998811 0.11001189]
[0.92606002 0.07393998]
[0.90527637 0.09472363]
[0.84831244 0.15168756]
[0.82188945 0.17811055]
[0.86351424 0.13648576]
[0.80273277 0.19726723]
[0.80577163 0.19422837]
[0.00995025 0.99004975]
[0.8970914 0.1029086]
[0.80733181 0.19266819]
[0.7314919 0.2685081]
[0.90340663 0.09659337]
[0.83470195 0.16529805]
[0.88844644 0.11155356]
[0.8632729 0.1367271]
[0.76801061 0.23198939]
[0.84689782 0.15310218]
[0.88711265 0.11288735]
[0.87686501 0.12313499]
[0.07949414 0.92050586]
[0.84226265 0.15773735]
[0.84542745 0.15457255]
[0.82671922 0.17328078]
[0.83624697 0.16375303]
[0.49675629 0.50324371]
[0.84959147 0.15040853]
[0. 1.]
[0.91023925 0.0897

[0.83406283 0.16593717]
[0.85267153 0.14732847]
[0.88483582 0.11516418]
[0.85368103 0.14631897]
[0.88712763 0.11287237]
[0.88437557 0.11562443]
[0.86023717 0.13976283]
[0.91222084 0.08777916]
[0.89186165 0.10813835]
[0.89043254 0.10956746]
[0.87550526 0.12449474]
[0.89744489 0.10255511]
[0.89627649 0.10372351]
[0.84411066 0.15588934]
[0.81036735 0.18963265]
[0.86590856 0.13409144]
[0.82651932 0.17348068]
[0.81184738 0.18815262]
[0.83496097 0.16503903]
[0.70182409 0.29817591]
[0.83714839 0.16285161]
[0.89175522 0.10824478]
[0.90046859 0.09953141]
[0.85958363 0.14041637]
[0.88693134 0.11306866]
[0.8299221 0.1700779]
[0.86663156 0.13336844]
[0.85303792 0.14696208]
[0.82985444 0.17014556]
[0.86296235 0.13703765]
[0.86716087 0.13283913]
[0.88081746 0.11918254]
[0.06296451 0.93703549]
[0.86637845 0.13362155]
[0.86929971 0.13070029]
[0.83882027 0.16117973]
[0.89562397 0.10437603]
[0.85009614 0.14990386]
[0.82373163 0.17626837]
[0.87217803 0.12782197]
[0.80990665 0.19009335]
[0.88140997 0.1185

[0.6147173 0.3852827]
[0.86345555 0.13654445]
[0.88034034 0.11965966]
[0.87485526 0.12514474]
[0.84006796 0.15993204]
[0.8612133 0.1387867]
[0.86538377 0.13461623]
[0.86825576 0.13174424]
[0.85852655 0.14147345]
[0.86069431 0.13930569]
[0.90415422 0.09584578]
[0.90966861 0.09033139]
[0.90648968 0.09351032]
[0.86552718 0.13447282]
[0.02966827 0.97033173]
[0.88087284 0.11912716]
[0.90378114 0.09621886]
[0.88829922 0.11170078]
[0.75299531 0.24700469]
[0.89419632 0.10580368]
[0.84021059 0.15978941]
[0.0152097 0.9847903]
[0.85029339 0.14970661]
[0.83958602 0.16041398]
[0.79296124 0.20703876]
[0.9076697 0.0923303]
[0.85268695 0.14731305]
[0.78947593 0.21052407]
[0.89482635 0.10517365]
[0.82267875 0.17732125]
[0.44508132 0.55491868]
[0.68029717 0.31970283]
[0.90290003 0.09709997]
[0.82735199 0.17264801]
[0.8255855 0.1744145]
[0.90011014 0.09988986]
[0.9003923 0.0996077]
[0.8142939 0.1857061]
[0.80147145 0.19852855]
[0.81689461 0.18310539]
[0.83445751 0.16554249]
[0.83244288 0.16755712]
[0.908

...

[0.90098352 0.09901648]
[0.91973744 0.08026256]
[0.90104234 0.09895766]
[0.03619638 0.96380362]
[0.8752607 0.1247393]
[0.89224473 0.10775527]
[0.85962335 0.14037665]
[0.90047216 0.09952784]
[0.83093767 0.16906233]
[0.84597855 0.15402145]
[0.86870814 0.13129186]
[0.8762486 0.1237514]
[0.13120839 0.86879161]
[0.86789616 0.13210384]
[0.8283525 0.1716475]
[0.67572118 0.32427882]
[0.88159461 0.11840539]
[0.89371549 0.10628451]
[0.90424496 0.09575504]
[0.89235611 0.10764389]
[0.91422614 0.08577386]
[0.83733407 0.16266593]
[0.00497512 0.99502488]
[0.85918057 0.14081943]
[0.82181241 0.17818759]
[0.84673127 0.15326873]
[0.83052629 0.16947371]
[0.88489007 0.11510993]
[0.76362939 0.23637061]
[0.85967165 0.14032835]
[0.85705662 0.14294338]
[0.88452573 0.11547427]
[0.89578474 0.10421526]
[0.80479785 0.19520215]
[0.1851226 0.8148774]
[0.85272167 0.14727833]
[0.62927912 0.37072088]
[0.86949213 0.13050787]
[0.89663211 0.10336789]
[0.9167572 0.0832428]
[0.6405953 0.3594047]
[0.92921529 0.07078471]
[0.9

[0.7746519 0.2253481]
[0.88891772 0.11108228]
[0.84856696 0.15143304]
[0.85830409 0.14169591]
[0.85696696 0.14303304]
[0.00497512 0.99502488]
[0.70634453 0.29365547]
[0.86802786 0.13197214]
[0.09580824 0.90419176]
[0.05343854 0.94656146]
[0.83965689 0.16034311]
[0.85753615 0.14246385]
[0.86616886 0.13383114]
[0.01047477 0.98952523]
[0.73245029 0.26754971]
[0.86327317 0.13672683]
[0.91121745 0.08878255]
[0.86895663 0.13104337]
[0.84582199 0.15417801]
[0.88872473 0.11127527]
[0.7540216 0.2459784]
[0.88298572 0.11701428]
[0.88430729 0.11569271]
[0.9012743 0.0987257]
[0.89959639 0.10040361]
[0.88490941 0.11509059]
[0.25584418 0.74415582]
[0.85924268 0.14075732]
[0.90155895 0.09844105]
[0.85790911 0.14209089]
[0.89203627 0.10796373]
[0.84705101 0.15294899]
[0.77882087 0.22117913]
[0.87790365 0.12209635]
[0.90222437 0.09777563]
[0.64552673 0.35447327]
[0.88201279 0.11798721]
[0.80373115 0.19626885]
[0.8886334 0.1113666]
[0.89815115 0.10184885]
[0.81927232 0.18072768]
[0.8546815 0.1453185]
[0

[0.93029879 0.06970121]
[0.86590403 0.13409597]
[0.87158169 0.12841831]
[0.85893139 0.14106861]
[0.83766786 0.16233214]
[0.8846358 0.1153642]
[0.87420825 0.12579175]
[0.86265039 0.13734961]
[0.84101019 0.15898981]
[0.85963966 0.14036034]
[0.85058876 0.14941124]
[0.86669285 0.13330715]
[0.87620492 0.12379508]
[0.82776383 0.17223617]
[0.87026116 0.12973884]
[0.06533472 0.93466528]
[0.86580769 0.13419231]
[0.85278086 0.14721914]
[0.89778745 0.10221255]
[0.84765836 0.15234164]
[0.82843747 0.17156253]
[0.83115226 0.16884774]
[0.80643151 0.19356849]
[0.8991434 0.1008566]
[0.88104334 0.11895666]
[0.91191696 0.08808304]
[0.86431423 0.13568577]
[0.9036646 0.0963354]
[0.87316263 0.12683737]
[0.84990735 0.15009265]
[0.87487912 0.12512088]
[0.90490536 0.09509464]
[0.83679828 0.16320172]
[0.85207426 0.14792574]
[0.84676177 0.15323823]
[0.89908319 0.10091681]
[0.86908152 0.13091848]
[0.70109996 0.29890004]
[0.88363612 0.11636388]
[0.96170475 0.03829525]
[0.84472076 0.15527924]
[0.74707753 0.25292247

[0.89338245 0.10661755]
[0.87250673 0.12749327]
[0.90595316 0.09404684]
[0.64984679 0.35015321]
[0.8575897 0.1424103]
[0.85352157 0.14647843]
[0.87054701 0.12945299]
[0.91527361 0.08472639]
[0.69931514 0.30068486]
[0.68036927 0.31963073]
[0.91644197 0.08355803]
[0.88494281 0.11505719]
[0.8574655 0.1425345]
[0.85942742 0.14057258]
[0.88753198 0.11246802]
[0.84172592 0.15827408]
[0.73179856 0.26820144]
[0.88006545 0.11993455]
[0.89097089 0.10902911]
[0.92461986 0.07538014]
[0.9071546 0.0928454]
[0.89585539 0.10414461]
[0.92331608 0.07668392]
[0.08558044 0.91441956]
[0.88589222 0.11410778]
[0.89076275 0.10923725]
[0.84598291 0.15401709]
[0.85524411 0.14475589]
[0.86707308 0.13292692]
[0.81478548 0.18521452]
[0.85944869 0.14055131]
[0.84675613 0.15324387]
[0.90159938 0.09840062]
[0.85780573 0.14219427]
[0.78986669 0.21013331]
[0.84966475 0.15033525]
[0.66178186 0.33821814]
[0.76682574 0.23317426]
[0.8553518 0.1446482]
[0.83345368 0.16654632]
[0.89085962 0.10914038]
[0.75855906 0.24144094]


[0.85325037 0.14674963]
[0.86350248 0.13649752]
[0.87140467 0.12859533]
[0.83516447 0.16483553]
[0.90115288 0.09884712]
[0.90511481 0.09488519]
[0.88974247 0.11025753]
[0.03224828 0.96775172]
[0.89142272 0.10857728]
[0.87535546 0.12464454]
[0.87492252 0.12507748]
[0.83047647 0.16952353]
[0.85359094 0.14640906]
[0.84314066 0.15685934]
[0.83064911 0.16935089]
[0.5407692 0.4592308]
[0.90483237 0.09516763]
[0.8958507 0.1041493]
[0.83603173 0.16396827]
[0.86277216 0.13722784]
[0.90873581 0.09126419]
[0.6227035 0.3772965]
[0.88112769 0.11887231]
[0.88591913 0.11408087]
[0.809913 0.190087]
[0.8856313 0.1143687]
[0.00497512 0.99502488]
[0.86117647 0.13882353]
[0.90184911 0.09815089]
[0.84030846 0.15969154]
[0.89403536 0.10596464]
[0.84445213 0.15554787]
[0.7404579 0.2595421]
[0.86632699 0.13367301]
[0.8765866 0.1234134]
[0.90165925 0.09834075]
[0.87411289 0.12588711]
[0.82789795 0.17210205]
[0.80605967 0.19394033]
[0.50190401 0.49809599]
[0.83004414 0.16995586]
[0.86972348 0.13027652]
[0.00574

### Very good

### Writing model into file

In [49]:
import pickle
filename = "Random_Forest.sav"
pickle.dump(rf, open(filename, "wb"))

### Loading from model

In [50]:
import pickle
filename = "Random_Forest.sav"
loaded_model = pickle.load(open(filename, "rb"))

In [51]:
accuracy_score(y_test, loaded_model.predict(X_test))

0.9686390532544379

# KNN

In [52]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier(n_neighbors = 23, n_jobs = -1)

# params = {
#     "n_neighbors": np.arange(7, 14, 2),
# }
# knn = GridSearchCV(knn, params, n_jobs = -1, verbose = 10)

In [53]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=23, p=2,
                     weights='uniform')

In [55]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
accuracy_score(y_test, knn.predict(X_test)), recall_score(y_test, knn.predict(X_test)), precision_score(y_test, knn.predict(X_test))

(0.9617497886728656, 0.5532022188603126, 0.9829749103942652)

# SVM

In [56]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svm = SVC(verbose = 10, random_state = 11)

# params = {
#     "C": [10, 1, 0.1, 0.01],
#     "degree": np.arange(2, 5)
# }

# svm = GridSearchCV(svm, params, verbose = 10, n_jobs = -1)

In [57]:
svm.fit(X_train, y_train)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=11, shrinking=True, tol=0.001,
    verbose=10)

In [58]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
accuracy_score(y_test, svm.predict(X_test)), recall_score(y_test, svm.predict(X_test)), \
         precision_score(y_test, svm.predict(X_test))

(0.9530008453085376, 0.45738779626828036, 0.9618239660657476)

# XGBoost

In [42]:
import xgboost as xgb
from xgboost import XGBClassifier

In [43]:
dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test)

In [44]:
dtrain

In [58]:
params = {'booster': 'gbtree', 'max_depth': 3, 'eta': 0.1, 
    'silent': 1, 'objective': 'binary:logistic', 'nthread': 1}
num_round = 2

bst = xgb.train(params, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

In [55]:
preds

array([0.05759477, 0.05759477, 0.05759477, ..., 0.05759477, 0.05759477,
       0.05759477], dtype=float32)

# XGBClassifier

In [59]:
from xgboost import XGBClassifier

params = {'booster': 'gbtree', 'max_depth': 3, 'eta': 0.1, 
    'silent': 1, 'objective': 'binary:logistic', 'nthread': 1, "learning_rate": 0.1}

model = XGBClassifier(**params)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)

In [60]:
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print("Общая точность: {}".format(accuracy))
print("Полнота (определение больного человека): {}".format(recall))
print("Точность положительного диагноза: {}".format(precision))

Общая точность: 0.9732882502113271
Полнота (определение больного человека): 0.761976802824004
Точность положительного диагноза: 0.9042489527229204


In [61]:
total = 0
middle = 0
for elem in model.predict_proba(X_test):
    if 0.25 <= elem[0] <= 0.75:
        middle += 1
        print(elem)
    total += 1

[0.409518 0.590482]
[0.49552578 0.5044742 ]
[0.25432783 0.74567217]
[0.40223688 0.5977631 ]
[0.5406384 0.4593616]
[0.27922714 0.72077286]
[0.4512546 0.5487454]
[0.31050843 0.68949157]
[0.6769535 0.3230465]
[0.49421072 0.5057893 ]
[0.4794572 0.5205428]
[0.36621922 0.6337808 ]
[0.25432783 0.74567217]
[0.3880093 0.6119907]
[0.68352115 0.31647888]
[0.67994475 0.32005528]
[0.39127022 0.6087298 ]
[0.3880093 0.6119907]
[0.58544934 0.41455063]
[0.48699725 0.51300275]
[0.6769535 0.3230465]
[0.48288018 0.5171198 ]
[0.39127022 0.6087298 ]
[0.67994475 0.32005528]
[0.48288018 0.5171198 ]
[0.6769535 0.3230465]
[0.4794572 0.5205428]
[0.6769535 0.3230465]
[0.6878122  0.31218782]
[0.3880093 0.6119907]
[0.3952031 0.6047969]
[0.3880093 0.6119907]
[0.68129206 0.31870794]
[0.28815746 0.71184254]
[0.39127022 0.6087298 ]
[0.3880093 0.6119907]
[0.4794572 0.5205428]
[0.68352115 0.31647888]
[0.48288018 0.5171198 ]
[0.40223688 0.5977631 ]
[0.6878122  0.31218782]
[0.40929854 0.59070146]
[0.4878533 0.5121467]
[0.6

[0.30819362 0.6918064 ]
[0.50268865 0.49731135]
[0.3952031 0.6047969]
[0.50889325 0.49110675]
[0.68426186 0.31573814]
[0.39127022 0.6087298 ]
[0.3068893 0.6931107]
[0.48699725 0.51300275]
[0.2569369 0.7430631]
[0.6769535 0.3230465]
[0.3880093 0.6119907]
[0.67994475 0.32005528]
[0.6836673  0.31633267]
[0.2569369 0.7430631]
[0.39127022 0.6087298 ]
[0.3880093 0.6119907]
[0.40553814 0.59446186]
[0.6769535 0.3230465]
[0.48288018 0.5171198 ]
[0.3952031 0.6047969]
[0.3880093 0.6119907]
[0.25432783 0.74567217]
[0.6769535 0.3230465]
[0.39127022 0.6087298 ]
[0.3880093 0.6119907]
[0.39127022 0.6087298 ]
[0.67994475 0.32005528]
[0.48442805 0.51557195]
[0.67994475 0.32005528]
[0.68352115 0.31647888]
[0.49450058 0.5054994 ]
[0.71865296 0.28134704]
[0.39127022 0.6087298 ]
[0.6769535 0.3230465]
[0.25432783 0.74567217]
[0.68352115 0.31647888]
[0.68129206 0.31870794]
[0.4794572 0.5205428]
[0.68129206 0.31870794]
[0.40223688 0.5977631 ]
[0.48288018 0.5171198 ]
[0.3880093 0.6119907]
[0.68352115 0.31647888

In [62]:
print(total, middle)

23660 610


In [127]:
for elem in model.predict_proba(X_test):
    print(elem)

[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.9642347  0.03576532]
[0.9830115 0.0169885]
[0.98667884 0.01332115]
[0.9868878 0.0131122]
[0.9830115 0.0169885]
[0.9838201  0.01617994]
[4.6014786e-05 9.9995399e-01]
[0.9830702  0.01692983]
[0.9860478  0.01395221]
[1.4150143e-04 9.9985850e-01]
[0.9660216 0.0339784]
[0.9840506  0.01594942]
[0.9838201  0.01617994]
[0.98667884 0.01332115]
[0.9830702  0.01692983]
[0.10460979 0.8953902 ]
[0.9829373  0.01706273]
[0.9860478  0.01395221]
[0.98667884 0.01332115]
[0.9838201  0.01617994]
[0.9860478  0.01395221]
[0.9840506  0.01594942]
[0.9860478  0.01395221]
[0.984463 0.015537]
[0.9860478  0.01395221]
[0.98667884 0.01332115]
[0.9864858  0.01351421]
[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.9830702  0.01692983]
[0.9860478  0.01395221]
[0.9852584  0.01474163]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.9864858  0.01351421]
[0.9840506  0.01594942]
[0.9830115 0.0169885]
[0.9864858  0.0135

[0.9830702  0.01692983]
[0.98667884 0.01332115]
[0.9864858  0.01351421]
[0.98667884 0.01332115]
[0.96175283 0.03824719]
[0.9860478  0.01395221]
[0.33109283 0.66890717]
[0.9795861  0.02041389]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.9864858  0.01351421]
[0.9829373  0.01706273]
[0.9840506  0.01594942]
[0.9830702  0.01692983]
[0.9793995  0.02060052]
[0.9860478  0.01395221]
[0.9674144  0.03258562]
[0.98681265 0.01318733]
[0.98667884 0.01332115]
[0.9838201  0.01617994]
[0.9840506  0.01594942]
[0.9838201  0.01617994]
[0.00440437 0.99559563]
[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.9860478  0.01395221]
[0.9829373  0.01706273]
[0.9793995  0.02060052]
[0.98667884 0.01332115]
[0.9864858  0.01351421]
[0.9830702  0.01692983]
[0.9830702  0.01692983]
[0.9640766 0.0359234]
[0.9860478  0.01395221]
[0.9838201  0.01617994]
[0.9830702  0.01692983]
[0.9838201  0.01617994]
[0.9830115 0.0169885]
[0.9838201  0.01617994]
[0.98667884 0.01332115]
[0.9829373  0.01706273]
[0.9840506  0.015949

[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.9864858  0.01351421]
[0.9860478  0.01395221]
[0.98430014 0.01569989]
[0.9864858  0.01351421]
[0.00486171 0.9951383 ]
[0.9864858  0.01351421]
[0.9860478  0.01395221]
[0.96553963 0.03446037]
[0.9864858  0.01351421]
[0.9660216 0.0339784]
[0.9864858  0.01351421]
[0.9852584  0.01474163]
[0.9864858  0.01351421]
[0.9661003  0.03389973]
[0.98667884 0.01332115]
[0.9861879  0.01381214]
[0.9830702  0.01692983]
[0.9860478  0.01395221]
[0.9795861  0.02041389]
[0.5102783  0.48972172]
[0.9860478  0.01395221]
[0.98681265 0.01318733]
[0.9829373  0.01706273]
[0.7797826 0.2202174]
[0.98364615 0.01635382]
[0.9830702  0.01692983]
[0.9830702  0.01692983]
[0.54453474 0.45546526]
[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.9441035  0.05589651]
[0.9864858  0.01351421]
[0.9795861  0.02041389]
[0.9840506  0.01594942]
[0.27417558 0.7258244 ]
[0.9852584  0.01474163]
[0.9364965  0.06350353]
[0.9829373  0.01706273]
[0.9838201  0.016179

[0.9840506  0.01594942]
[0.9795861  0.02041389]
[0.9830702  0.01692983]
[0.9864858  0.01351421]
[0.9830702  0.01692983]
[0.9840506  0.01594942]
[0.9864858  0.01351421]
[0.9860478  0.01395221]
[0.96148086 0.03851914]
[0.9830702  0.01692983]
[0.9864858  0.01351421]
[0.9840506  0.01594942]
[0.9860478  0.01395221]
[0.98667884 0.01332115]
[0.9830702  0.01692983]
[0.9840506  0.01594942]
[0.9795861  0.02041389]
[0.9864858  0.01351421]
[0.98667884 0.01332115]
[0.4604379 0.5395621]
[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.9829373  0.01706273]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.9846187  0.01538126]
[0.9838201  0.01617994]
[0.98667884 0.01332115]
[0.9397809  0.06021913]
[0.9840506  0.01594942]
[0.9830702  0.01692983]
[0.97884864 0.02115138]
[0.9830115 0.0169885]
[0.9864858  0.01351421]
[0.9829373  0.01706273]
[0.9795861  0.02041389]
[0.9830702  0.01692983]
[0.6538223  0.34617773]
[0.9830702  0.01692983]
[0.9838201  0.01617994]
[0.9793995  0.020600

[0.86984384 0.13015614]
[0.9830702  0.01692983]
[0.9810912 0.0189088]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.9612127  0.03878728]
[0.9838201  0.01617994]
[0.98667884 0.01332115]
[0.9640766 0.0359234]
[0.9830115 0.0169885]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.9830702  0.01692983]
[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.9830702  0.01692983]
[0.98667884 0.01332115]
[5.596280e-04 9.994404e-01]
[0.9838201  0.01617994]
[0.98667884 0.01332115]
[0.9838201  0.01617994]
[0.9860478  0.01395221]
[0.9792338 0.0207662]
[0.9795861  0.02041389]
[0.9860478  0.01395221]
[0.9860478  0.01395221]
[0.9838201  0.01617994]
[1.2934208e-04 9.9987066e-01]
[0.9838201  0.01617994]
[0.9830115 0.0169885]
[0.9864858  0.01351421]
[0.9829373  0.01706273]
[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.95931786 0.04068215]
[0.9864858  0.01351421]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.9661003  0.03389973]
[0.9830702  0.01

...

[0.9864858  0.01351421]
[0.9840506  0.01594942]
[0.9793995  0.02060052]
[0.9642347  0.03576532]
[0.98667884 0.01332115]
[0.9566672  0.04333281]
[0.9660216 0.0339784]
[0.9840506  0.01594942]
[0.9860478  0.01395221]
[0.9864858  0.01351421]
[0.9860478  0.01395221]
[0.9840506  0.01594942]
[0.9829373  0.01706273]
[0.9507873  0.04921267]
[0.98667884 0.01332115]
[0.98667884 0.01332115]
[0.9860478  0.01395221]
[0.0123055 0.9876945]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.9838201  0.01617994]
[0.9860478  0.01395221]
[0.9830702  0.01692983]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.9793995  0.02060052]
[0.0794459 0.9205541]
[0.9829373  0.01706273]
[0.967871 0.032129]
[0.9368111  0.06318889]
[0.9860478  0.01395221]
[0.9864858  0.01351421]
[0.9829373  0.01706273]
[0.9864858  0.01351421]
[0.98667884 0.01332115]
[0.9830702  0.01692983]
[3.8540363e-04 9.9961460e-01]
[0.9830702  0.01692983]
[0.9830702  0.01692983]
[0.9840506  0.01594942]
[0.9830702  0.01692983]
[0.9864858  0.013514

[0.9829373  0.01706273]
[0.98364615 0.01635382]
[0.98667884 0.01332115]
[0.09876424 0.90123576]
[0.98667884 0.01332115]
[0.9860478  0.01395221]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.9786554  0.02134461]
[0.9313129  0.06868705]
[0.98681265 0.01318733]
[0.9864858  0.01351421]
[0.9577692  0.04223079]
[0.9840506  0.01594942]
[0.97884864 0.02115138]
[0.9860478  0.01395221]
[0.9864858  0.01351421]
[0.96635586 0.03364412]
[0.9830115 0.0169885]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.9860478  0.01395221]
[0.98681265 0.01318733]
[0.9838201  0.01617994]
[0.98667884 0.01332115]
[0.9830702  0.01692983]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.9830702  0.01692983]
[0.9860478  0.01395221]
[0.9829373  0.01706273]
[0.9868878 0.0131122]
[0.9860478  0.01395221]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.98231864 0.01768133]
[0.9829373  0.01706273]
[0.9829373  0.01706273]
[0.9829373  0.01706273]
[0.9838201  0.01617994]
[0.9860478  0.013952

[0.9838201  0.01617994]
[0.9795861  0.02041389]
[0.98667884 0.01332115]
[0.9548702  0.04512981]
[0.9838201  0.01617994]
[0.9830702  0.01692983]
[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.93515366 0.06484632]
[0.9795861  0.02041389]
[0.9710694  0.02893061]
[0.9838201  0.01617994]
[0.95931786 0.04068215]
[0.9840506  0.01594942]
[0.9829373  0.01706273]
[0.9852584  0.01474163]
[0.9829373  0.01706273]
[0.98667884 0.01332115]
[0.9830702  0.01692983]
[0.9852584  0.01474163]
[0.9830702  0.01692983]
[0.9852584  0.01474163]
[0.9830702  0.01692983]
[0.9861879  0.01381214]
[0.9840506  0.01594942]
[0.95726776 0.04273223]
[0.9838201  0.01617994]
[0.9795861  0.02041389]
[0.98667884 0.01332115]
[0.650514 0.349486]
[4.6873093e-04 9.9953127e-01]
[0.9840506  0.01594942]
[0.00753278 0.9924672 ]
[0.9840506  0.01594942]
[0.9838201  0.01617994]
[0.98667884 0.01332115]
[0.95726776 0.04273223]
[0.9838201  0.01617994]
[0.98667884 0.01332115]
[0.9864858  0.01351421]
[0.9840506  0.01594942]
[0.9860478  0.

[0.98667884 0.01332115]
[0.9830702  0.01692983]
[0.9860478  0.01395221]
[0.9710694  0.02893061]
[0.9864858  0.01351421]
[0.9830702  0.01692983]
[0.98667884 0.01332115]
[0.9583552  0.04164482]
[0.9795861  0.02041389]
[0.9838201  0.01617994]
[0.9795861  0.02041389]
[0.9861879  0.01381214]
[0.9642347  0.03576532]
[0.92067635 0.07932363]
[0.98430014 0.01569989]
[0.9860478  0.01395221]
[0.9840506  0.01594942]
[0.9830702  0.01692983]
[0.9864858  0.01351421]
[0.97884864 0.02115138]
[0.9170869  0.08291309]
[0.9830702  0.01692983]
[0.98667884 0.01332115]
[0.98667884 0.01332115]
[0.9864858  0.01351421]
[0.9860478  0.01395221]
[0.98430014 0.01569989]
[0.07473773 0.9252623 ]
[0.9860478  0.01395221]
[0.98667884 0.01332115]
[0.9830702  0.01692983]
[0.9840506  0.01594942]
[0.9795861  0.02041389]
[0.9830702  0.01692983]
[0.94015414 0.05984588]
[0.9840506  0.01594942]
[0.9864858  0.01351421]
[0.9637283  0.03627168]
[0.9795861  0.02041389]
[0.9840506  0.01594942]
[0.92562383 0.07437614]
[0.96175283 0.03

[0.9864858  0.01351421]
[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.94352555 0.05647442]
[0.98667884 0.01332115]
[0.9852584  0.01474163]
[0.9838201  0.01617994]
[0.98667884 0.01332115]
[0.00565791 0.9943421 ]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.9840506  0.01594942]
[0.9830115 0.0169885]
[0.9840506  0.01594942]
[0.98667884 0.01332115]
[0.9838201  0.01617994]
[0.9793995  0.02060052]
[0.9793995  0.02060052]
[0.9838201  0.01617994]
[0.98667884 0.01332115]
[0.9838201  0.01617994]
[0.98681265 0.01318733]
[0.9860478  0.01395221]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.9840506  0.01594942]
[0.9864858  0.01351421]
[0.3337741 0.6662259]
[0.9793995  0.02060052]
[0.9830702  0.01692983]
[0.9864858  0.01351421]
[0.9830702  0.01692983]
[2.1153688e-04 9.9978846e-01]
[0.9795861  0.02041389]
[0.9860478  0.01395221]
[0.9852584  0.01474163]
[0.9860478  0.01395221]
[0.9823955  0.01760446]
[0.9830702  0.01692983]
[0.00565791 0.9943421 ]
[0.9860478  0.01395221]
[0.9656194  0.

In [217]:
lst = [[2], [1], [2], [1], [0], [1], [1], [0]]

names = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'age', 'gender', 'test_indication']
lst = dict(zip(names, lst))

lst = pd.DataFrame(lst)
lst
print(model.predict_proba(lst))
print(rf.predict_proba(lst))

[[0.9852584  0.01474163]]
[[0.88076332 0.11923668]]


# Deep Net

### Reading csv

In [223]:
import torch
import torchvision
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import helper

In [224]:
X_train_tensor = torch.tensor(X_train.values)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
# dtype=torch.long

### Net

In [247]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, hidden_layers = 32, hidden_size = 32):
        super().__init__()
        self.fc1 = nn.Linear(8, hidden_size)
        self.layers = [nn.Linear(hidden_size, hidden_size)] * hidden_layers
        self.fc12 = nn.Linear(hidden_size, 2)
        
    def forward(self, X):
        X = X.view(-1, 8).float()
        
        X = F.relu(self.fc1(X))
        for layer in self.layers:
            X = F.relu(layer(X))
            
        X = self.fc12(X)
        
        return F.log_softmax(X, dim = 1)

In [332]:
import torch.optim as optim
net = Net(hidden_layers = 2, hidden_size = 16)

In [335]:
optimizer = optim.Adam(net.parameters(), lr = 0.2)
# optimizer = optim.SGD(net.parameters(), lr = 0.34, momentum = 0.038)

In [336]:
%%time
from tqdm import tqdm
EPOCHS = 100

for epoch in tqdm(range(EPOCHS)):
#     for i in range(0, len(X_train_tensor), 64):
    X = X_train_tensor
    y = y_train_tensor
    net.zero_grad()
    output = torch.FloatTensor(net(X))
    loss = F.nll_loss(output, y)
    loss.backward()
    optimizer.step()
    print(loss)

  2%|█                                                   | 2/100 [00:00<00:10,  8.97it/s]

tensor(0.2867, grad_fn=<NllLossBackward>)
tensor(0.3233, grad_fn=<NllLossBackward>)


  4%|██                                                  | 4/100 [00:00<00:10,  8.84it/s]

tensor(0.2889, grad_fn=<NllLossBackward>)
tensor(0.2920, grad_fn=<NllLossBackward>)


  6%|███                                                 | 6/100 [00:00<00:10,  9.03it/s]

...

tensor(0.2866, grad_fn=<NllLossBackward>)
tensor(0.2868, grad_fn=<NllLossBackward>)


 98%|█████████████████████████████████████████████████▉ | 98/100 [00:11<00:00,  8.53it/s]

tensor(0.2866, grad_fn=<NllLossBackward>)
tensor(0.2870, grad_fn=<NllLossBackward>)


100%|██████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.27it/s]

tensor(0.2871, grad_fn=<NllLossBackward>)
tensor(0.2871, grad_fn=<NllLossBackward>)
Wall time: 12.1 s


In [326]:
X_test_tensor = torch.tensor(X_test.values)

result = net(X_test_tensor)
print(torch.argmax(result))
# lst = [int(torch.argmax(elem)) for elem in result]
lst = [1 if elem[1] > -1.2 else int(torch.argmax(elem)) for elem in result]    ## -1.39; -1.89

print(set(lst))
print(torch.exp(result))

tensor(44645)
{0, 1}
tensor([[0.9907, 0.0093],
        [0.9893, 0.0107],
        [0.9358, 0.0642],
        ...,
        [0.9842, 0.0158],
        [0.9883, 0.0117],
        [0.9867, 0.0133]], grad_fn=<ExpBackward>)


### all the metrics are pretty nice having such a dataset

In [327]:
from sklearn.metrics import recall_score, accuracy_score, precision_score
accuracy_score(y_test, lst), recall_score(y_test, lst), \
            precision_score(y_test, lst)

(0.9664835164835165, 0.7536895674300255, 0.8273743016759777)

In [329]:
total = 0
middle = 0
for elem in torch.exp(result):
    if 0.25 <= elem[0] <= 0.75:
        middle += 1
        print(elem)
    total += 1

tensor([0.7088, 0.2912], grad_fn=<SelectBackward>)
tensor([0.2906, 0.7094], grad_fn=<SelectBackward>)
tensor([0.7397, 0.2603], grad_fn=<SelectBackward>)
tensor([0.4158, 0.5842], grad_fn=<SelectBackward>)
tensor([0.2963, 0.7037], grad_fn=<SelectBackward>)
tensor([0.6669, 0.3331], grad_fn=<SelectBackward>)
tensor([0.7423, 0.2577], grad_fn=<SelectBackward>)
tensor([0.2849, 0.7151], grad_fn=<SelectBackward>)
tensor([0.6865, 0.3135], grad_fn=<SelectBackward>)
tensor([0.4361, 0.5639], grad_fn=<SelectBackward>)
tensor([0.5396, 0.4604], grad_fn=<SelectBackward>)
tensor([0.6823, 0.3177], grad_fn=<SelectBackward>)
tensor([0.3917, 0.6083], grad_fn=<SelectBackward>)
tensor([0.6694, 0.3306], grad_fn=<SelectBackward>)
tensor([0.3456, 0.6544], grad_fn=<SelectBackward>)
tensor([0.7432, 0.2568], grad_fn=<SelectBackward>)
tensor([0.3078, 0.6922], grad_fn=<SelectBackward>)
tensor([0.4319, 0.5681], grad_fn=<SelectBackward>)
tensor([0.5459, 0.4541], grad_fn=<SelectBackward>)
tensor([0.5975, 0.4025], grad_f

tensor([0.7236, 0.2764], grad_fn=<SelectBackward>)
tensor([0.5499, 0.4501], grad_fn=<SelectBackward>)
tensor([0.6131, 0.3869], grad_fn=<SelectBackward>)
tensor([0.3796, 0.6204], grad_fn=<SelectBackward>)
tensor([0.3749, 0.6251], grad_fn=<SelectBackward>)
tensor([0.4218, 0.5782], grad_fn=<SelectBackward>)
tensor([0.6951, 0.3049], grad_fn=<SelectBackward>)
tensor([0.5885, 0.4115], grad_fn=<SelectBackward>)
tensor([0.4950, 0.5050], grad_fn=<SelectBackward>)
tensor([0.4833, 0.5167], grad_fn=<SelectBackward>)
tensor([0.3627, 0.6373], grad_fn=<SelectBackward>)
tensor([0.7390, 0.2610], grad_fn=<SelectBackward>)
tensor([0.4259, 0.5741], grad_fn=<SelectBackward>)
tensor([0.3343, 0.6657], grad_fn=<SelectBackward>)
tensor([0.4435, 0.5565], grad_fn=<SelectBackward>)
tensor([0.6544, 0.3456], grad_fn=<SelectBackward>)
tensor([0.7252, 0.2748], grad_fn=<SelectBackward>)
tensor([0.3943, 0.6057], grad_fn=<SelectBackward>)
tensor([0.7234, 0.2766], grad_fn=<SelectBackward>)
tensor([0.5024, 0.4976], grad_f

tensor([0.7458, 0.2542], grad_fn=<SelectBackward>)
tensor([0.3500, 0.6500], grad_fn=<SelectBackward>)
tensor([0.4903, 0.5097], grad_fn=<SelectBackward>)
tensor([0.4559, 0.5441], grad_fn=<SelectBackward>)
tensor([0.5156, 0.4844], grad_fn=<SelectBackward>)
tensor([0.5930, 0.4070], grad_fn=<SelectBackward>)
tensor([0.3673, 0.6327], grad_fn=<SelectBackward>)
tensor([0.6002, 0.3998], grad_fn=<SelectBackward>)
tensor([0.4615, 0.5385], grad_fn=<SelectBackward>)
tensor([0.6346, 0.3654], grad_fn=<SelectBackward>)
tensor([0.4297, 0.5703], grad_fn=<SelectBackward>)
tensor([0.4820, 0.5180], grad_fn=<SelectBackward>)
tensor([0.3834, 0.6166], grad_fn=<SelectBackward>)
tensor([0.4438, 0.5562], grad_fn=<SelectBackward>)
tensor([0.5156, 0.4844], grad_fn=<SelectBackward>)
tensor([0.3142, 0.6858], grad_fn=<SelectBackward>)
tensor([0.3465, 0.6535], grad_fn=<SelectBackward>)
tensor([0.6163, 0.3837], grad_fn=<SelectBackward>)
tensor([0.2690, 0.7310], grad_fn=<SelectBackward>)
tensor([0.5194, 0.4806], grad_f

tensor([0.7137, 0.2863], grad_fn=<SelectBackward>)
tensor([0.3876, 0.6124], grad_fn=<SelectBackward>)
tensor([0.3730, 0.6270], grad_fn=<SelectBackward>)
tensor([0.5571, 0.4429], grad_fn=<SelectBackward>)
tensor([0.2741, 0.7259], grad_fn=<SelectBackward>)
tensor([0.7253, 0.2747], grad_fn=<SelectBackward>)
tensor([0.2790, 0.7210], grad_fn=<SelectBackward>)
tensor([0.7183, 0.2817], grad_fn=<SelectBackward>)
tensor([0.3792, 0.6208], grad_fn=<SelectBackward>)
tensor([0.4356, 0.5644], grad_fn=<SelectBackward>)
tensor([0.7048, 0.2952], grad_fn=<SelectBackward>)
tensor([0.5294, 0.4706], grad_fn=<SelectBackward>)
tensor([0.4058, 0.5942], grad_fn=<SelectBackward>)
tensor([0.5462, 0.4538], grad_fn=<SelectBackward>)
tensor([0.5915, 0.4085], grad_fn=<SelectBackward>)
tensor([0.3648, 0.6352], grad_fn=<SelectBackward>)
tensor([0.3216, 0.6784], grad_fn=<SelectBackward>)
tensor([0.5930, 0.4070], grad_fn=<SelectBackward>)
tensor([0.5456, 0.4544], grad_fn=<SelectBackward>)
tensor([0.5985, 0.4015], grad_f

tensor([0.4371, 0.5629], grad_fn=<SelectBackward>)
tensor([0.5126, 0.4874], grad_fn=<SelectBackward>)
tensor([0.4231, 0.5769], grad_fn=<SelectBackward>)
tensor([0.2651, 0.7349], grad_fn=<SelectBackward>)
tensor([0.4822, 0.5178], grad_fn=<SelectBackward>)
tensor([0.3622, 0.6378], grad_fn=<SelectBackward>)
tensor([0.7082, 0.2918], grad_fn=<SelectBackward>)
tensor([0.5341, 0.4659], grad_fn=<SelectBackward>)
tensor([0.5244, 0.4756], grad_fn=<SelectBackward>)
tensor([0.2789, 0.7211], grad_fn=<SelectBackward>)
tensor([0.4193, 0.5807], grad_fn=<SelectBackward>)
tensor([0.4762, 0.5238], grad_fn=<SelectBackward>)
tensor([0.2985, 0.7015], grad_fn=<SelectBackward>)
tensor([0.4894, 0.5106], grad_fn=<SelectBackward>)
tensor([0.4432, 0.5568], grad_fn=<SelectBackward>)
tensor([0.6862, 0.3138], grad_fn=<SelectBackward>)
tensor([0.6698, 0.3302], grad_fn=<SelectBackward>)
tensor([0.7181, 0.2819], grad_fn=<SelectBackward>)
tensor([0.6742, 0.3258], grad_fn=<SelectBackward>)
tensor([0.4660, 0.5340], grad_f

In [330]:
print(total, middle)

23660 745


### checking each case

In [224]:
first = 0       ## ошибка первого рода, предсказать болезнь если здоров
second = 0      ## ошибка второго рода, предсказать здоровье в случае болезни
for num, i in enumerate(range(len(lst))):
    print(num, lst[i], y_test.iloc[i])
    if lst[i] == 1 and y_test.iloc[i] == 0:
        first += 1
    elif lst[i] == 0 and y_test.iloc[i] == 1:
        second += 1

0 0 0.0
1 1 1.0
2 0 1.0
3 0 0.0
4 0 0.0
5 0 0.0
6 0 0.0
7 0 0.0
8 0 0.0
9 0 0.0
10 0 1.0
11 0 0.0
12 0 0.0
13 0 0.0
14 0 0.0
15 0 0.0
16 0 0.0
17 0 0.0
18 0 0.0
19 0 0.0
20 0 0.0
21 0 0.0
22 0 0.0
23 0 0.0
24 1 1.0
25 1 1.0
26 0 0.0
27 0 0.0
28 0 0.0
29 0 0.0
30 0 1.0
31 0 0.0
32 0 0.0
33 0 0.0
34 0 0.0
35 0 0.0
36 0 0.0
37 0 0.0
38 0 0.0
39 0 0.0
40 0 0.0
41 0 0.0
42 0 0.0
43 0 0.0
44 0 0.0
45 0 0.0
46 0 0.0
47 1 1.0
48 0 0.0
49 0 0.0
50 0 0.0
51 0 0.0
52 0 0.0
53 0 0.0
54 0 0.0
55 0 0.0
56 0 0.0
57 0 0.0
58 1 1.0
59 0 0.0
60 0 0.0
61 0 0.0
62 0 1.0
63 0 0.0
64 0 0.0
65 0 0.0
66 0 0.0
67 0 0.0
68 0 0.0
69 0 0.0
70 0 0.0
71 0 0.0
72 0 0.0
73 0 0.0
74 0 0.0
75 0 0.0
76 0 0.0
77 0 0.0
78 0 0.0
79 0 0.0
80 0 0.0
81 0 0.0
82 0 0.0
83 0 0.0
84 0 0.0
85 0 0.0
86 0 0.0
87 0 0.0
88 0 0.0
89 0 0.0
90 0 0.0
91 1 1.0
92 0 0.0
93 0 1.0
94 0 0.0
95 0 1.0
96 0 0.0
97 0 0.0
98 0 0.0
99 0 0.0
100 0 0.0
101 0 0.0
102 0 0.0
103 0 0.0
104 0 0.0
105 0 0.0
106 0 0.0
107 0 0.0
108 0 0.0
109 0 0.0
110 0 0.0


1271 0 0.0
1272 0 0.0
1273 0 0.0
1274 0 0.0
1275 0 0.0
1276 1 1.0
1277 0 0.0
1278 0 1.0
1279 0 0.0
1280 0 0.0
1281 1 1.0
1282 0 0.0
1283 0 0.0
1284 0 0.0
1285 0 0.0
1286 1 1.0
1287 0 0.0
1288 0 0.0
1289 0 0.0
1290 0 0.0
1291 0 0.0
1292 0 0.0
1293 0 0.0
1294 1 1.0
1295 0 0.0
1296 0 0.0
1297 0 0.0
1298 0 1.0
1299 0 0.0
1300 0 0.0
1301 0 0.0
1302 0 0.0
1303 0 0.0
1304 0 0.0
1305 0 0.0
1306 0 0.0
1307 0 0.0
1308 0 0.0
1309 1 1.0
1310 0 0.0
1311 0 0.0
1312 0 0.0
1313 0 0.0
1314 0 0.0
1315 0 0.0
1316 0 0.0
1317 0 0.0
1318 0 0.0
1319 0 0.0
1320 1 0.0
1321 0 0.0
1322 0 0.0
1323 0 0.0
1324 0 0.0
1325 0 0.0
1326 0 0.0
1327 0 0.0
1328 0 0.0
1329 0 0.0
1330 0 0.0
1331 0 0.0
1332 0 0.0
1333 0 0.0
1334 0 0.0
1335 0 0.0
1336 0 0.0
1337 0 0.0
1338 0 0.0
1339 0 0.0
1340 1 1.0
1341 0 0.0
1342 0 0.0
1343 0 0.0
1344 0 0.0
1345 0 0.0
1346 0 0.0
1347 0 0.0
1348 0 0.0
1349 0 0.0
1350 0 1.0
1351 0 0.0
1352 1 1.0
1353 0 0.0
1354 1 1.0
1355 0 1.0
1356 0 0.0
1357 0 0.0
1358 0 0.0
1359 0 0.0
1360 0 0.0
1361 0 0.0

2735 0 0.0
2736 0 1.0
2737 0 0.0
2738 0 0.0
2739 0 0.0
2740 0 0.0
2741 0 0.0
2742 0 0.0
2743 0 0.0
2744 0 0.0
2745 0 1.0
2746 0 0.0
2747 0 0.0
2748 0 0.0
2749 0 0.0
2750 0 0.0
2751 0 0.0
2752 0 0.0
2753 0 0.0
2754 0 0.0
2755 0 0.0
2756 0 0.0
2757 0 0.0
2758 0 0.0
2759 0 0.0
2760 0 0.0
2761 0 0.0
2762 0 0.0
2763 0 0.0
2764 0 0.0
2765 0 0.0
2766 0 0.0
2767 0 0.0
2768 0 0.0
2769 0 0.0
2770 0 0.0
2771 0 0.0
2772 0 0.0
2773 0 0.0
2774 0 0.0
2775 0 0.0
2776 0 0.0
2777 0 0.0
2778 0 0.0
2779 0 0.0
2780 0 0.0
2781 0 0.0
2782 0 0.0
2783 0 0.0
2784 0 0.0
2785 0 0.0
2786 1 1.0
2787 0 0.0
2788 0 0.0
2789 0 0.0
2790 0 0.0
2791 0 0.0
2792 0 0.0
2793 1 0.0
2794 0 0.0
2795 0 0.0
2796 0 0.0
2797 1 0.0
2798 0 0.0
2799 0 0.0
2800 0 0.0
2801 0 0.0
2802 0 0.0
2803 0 0.0
2804 0 0.0
2805 0 0.0
2806 0 0.0
2807 1 1.0
2808 0 0.0
2809 0 0.0
2810 0 0.0
2811 0 0.0
2812 0 0.0
2813 0 0.0
2814 0 0.0
2815 0 0.0
2816 0 1.0
2817 0 0.0
2818 0 0.0
2819 0 0.0
2820 0 0.0
2821 0 0.0
2822 0 0.0
2823 0 0.0
2824 0 0.0
2825 0 0.0

3957 0 0.0
3958 0 0.0
3959 0 0.0
3960 0 0.0
3961 0 0.0
3962 0 0.0
3963 0 0.0
3964 0 0.0
3965 0 0.0
3966 1 1.0
3967 0 0.0
3968 0 0.0
3969 0 0.0
3970 0 0.0
3971 1 1.0
3972 0 0.0
3973 0 0.0
3974 1 0.0
3975 0 0.0
3976 0 0.0
3977 0 0.0
3978 0 0.0
3979 0 0.0
3980 0 0.0
3981 0 0.0
3982 0 0.0
3983 0 0.0
3984 0 0.0
3985 0 0.0
3986 0 0.0
3987 0 0.0
3988 0 0.0
3989 0 0.0
3990 0 0.0
3991 0 0.0
3992 0 0.0
3993 0 0.0
3994 0 0.0
3995 1 1.0
3996 0 0.0
3997 0 0.0
3998 0 0.0
3999 0 0.0
4000 0 0.0
4001 0 0.0
4002 0 0.0
4003 0 0.0
4004 0 0.0
4005 0 0.0
4006 0 0.0
4007 0 0.0
4008 0 0.0
4009 0 0.0
4010 0 0.0
4011 0 0.0
4012 0 0.0
4013 0 0.0
4014 0 0.0
4015 0 0.0
4016 0 0.0
4017 1 1.0
4018 0 0.0
4019 0 0.0
4020 1 0.0
4021 0 0.0
4022 0 0.0
4023 0 0.0
4024 0 0.0
4025 0 0.0
4026 0 0.0
4027 0 0.0
4028 0 0.0
4029 0 0.0
4030 0 1.0
4031 0 0.0
4032 0 0.0
4033 0 0.0
4034 0 0.0
4035 1 1.0
4036 0 0.0
4037 0 0.0
4038 0 0.0
4039 0 0.0
4040 0 0.0
4041 0 0.0
4042 0 0.0
4043 0 0.0
4044 0 0.0
4045 0 0.0
4046 0 0.0
4047 0 0.0

5188 0 0.0
5189 0 0.0
5190 0 0.0
5191 0 0.0
5192 0 0.0
5193 0 0.0
5194 1 1.0
5195 0 0.0
5196 0 0.0
5197 0 0.0
5198 0 0.0
5199 0 0.0
5200 0 0.0
5201 0 0.0
5202 0 0.0
5203 0 0.0
5204 0 0.0
5205 0 0.0
5206 0 0.0
5207 0 0.0
5208 0 0.0
5209 0 0.0
5210 0 0.0
5211 0 1.0
5212 0 1.0
5213 0 0.0
5214 1 1.0
5215 0 0.0
5216 0 0.0
5217 0 0.0
5218 0 0.0
5219 0 0.0
5220 0 0.0
5221 0 0.0
5222 0 0.0
5223 0 0.0
5224 1 1.0
5225 0 0.0
5226 0 0.0
5227 0 0.0
5228 0 0.0
5229 0 0.0
5230 0 0.0
5231 0 0.0
5232 0 0.0
5233 0 0.0
5234 0 0.0
5235 0 0.0
5236 1 1.0
5237 0 0.0
5238 0 0.0
5239 1 1.0
5240 0 0.0
5241 0 0.0
5242 0 0.0
5243 0 0.0
5244 0 0.0
5245 0 0.0
5246 1 1.0
5247 0 1.0
5248 0 0.0
5249 0 0.0
5250 0 0.0
5251 0 0.0
5252 0 0.0
5253 0 0.0
5254 0 0.0
5255 0 0.0
5256 1 1.0
5257 0 0.0
5258 0 0.0
5259 0 0.0
5260 1 1.0
5261 0 0.0
5262 0 0.0
5263 0 0.0
5264 0 0.0
5265 0 0.0
5266 0 0.0
5267 0 0.0
5268 0 0.0
5269 0 0.0
5270 0 0.0
5271 0 0.0
5272 0 0.0
5273 0 0.0
5274 0 0.0
5275 0 0.0
5276 0 0.0
5277 0 0.0
5278 1 0.0

...

19743 0 0.0
19744 0 0.0
19745 0 0.0
19746 0 0.0
19747 0 0.0
19748 0 0.0
19749 0 0.0
19750 0 0.0
19751 0 0.0
19752 0 0.0
19753 0 0.0
19754 0 0.0
19755 0 1.0
19756 1 1.0
19757 0 0.0
19758 0 0.0
19759 0 0.0
19760 0 0.0
19761 0 0.0
19762 0 0.0
19763 0 0.0
19764 1 0.0
19765 0 0.0
19766 0 0.0
19767 0 0.0
19768 0 0.0
19769 0 0.0
19770 1 1.0
19771 0 0.0
19772 0 0.0
19773 0 0.0
19774 0 0.0
19775 0 0.0
19776 0 0.0
19777 0 0.0
19778 0 0.0
19779 0 0.0
19780 0 0.0
19781 0 0.0
19782 0 0.0
19783 0 0.0
19784 0 0.0
19785 0 0.0
19786 0 0.0
19787 0 0.0
19788 1 1.0
19789 0 0.0
19790 0 0.0
19791 0 0.0
19792 0 1.0
19793 0 0.0
19794 0 0.0
19795 0 0.0
19796 1 1.0
19797 0 0.0
19798 0 0.0
19799 0 0.0
19800 0 0.0
19801 0 0.0
19802 0 0.0
19803 0 0.0
19804 0 0.0
19805 1 1.0
19806 0 0.0
19807 0 0.0
19808 0 0.0
19809 0 0.0
19810 0 0.0
19811 0 0.0
19812 0 0.0
19813 0 0.0
19814 0 0.0
19815 0 0.0
19816 0 0.0
19817 0 0.0
19818 0 0.0
19819 0 0.0
19820 0 0.0
19821 0 0.0
19822 0 0.0
19823 0 0.0
19824 1 1.0
19825 0 0.0
1982

20730 0 0.0
20731 0 0.0
20732 0 0.0
20733 0 0.0
20734 0 0.0
20735 1 0.0
20736 0 0.0
20737 0 0.0
20738 0 0.0
20739 0 0.0
20740 0 0.0
20741 0 0.0
20742 0 0.0
20743 0 0.0
20744 0 0.0
20745 0 0.0
20746 0 0.0
20747 0 0.0
20748 1 1.0
20749 0 0.0
20750 0 0.0
20751 0 0.0
20752 0 0.0
20753 0 0.0
20754 0 0.0
20755 0 0.0
20756 0 0.0
20757 0 0.0
20758 0 0.0
20759 0 0.0
20760 0 0.0
20761 0 0.0
20762 0 0.0
20763 0 0.0
20764 0 0.0
20765 0 0.0
20766 0 0.0
20767 0 0.0
20768 0 0.0
20769 0 0.0
20770 0 0.0
20771 0 0.0
20772 0 0.0
20773 0 0.0
20774 0 0.0
20775 0 0.0
20776 0 0.0
20777 0 0.0
20778 0 0.0
20779 0 0.0
20780 0 0.0
20781 0 0.0
20782 0 0.0
20783 0 0.0
20784 0 0.0
20785 0 0.0
20786 0 0.0
20787 0 0.0
20788 0 0.0
20789 0 0.0
20790 0 0.0
20791 0 0.0
20792 0 0.0
20793 0 0.0
20794 0 0.0
20795 0 0.0
20796 0 0.0
20797 0 0.0
20798 0 0.0
20799 0 0.0
20800 0 0.0
20801 0 0.0
20802 0 0.0
20803 0 0.0
20804 0 0.0
20805 1 0.0
20806 0 0.0
20807 0 0.0
20808 1 1.0
20809 0 0.0
20810 0 0.0
20811 0 0.0
20812 0 0.0
2081

21602 0 0.0
21603 0 0.0
21604 0 0.0
21605 0 0.0
21606 0 0.0
21607 0 0.0
21608 0 0.0
21609 0 0.0
21610 0 0.0
21611 0 0.0
21612 0 0.0
21613 0 0.0
21614 0 0.0
21615 0 0.0
21616 0 0.0
21617 0 0.0
21618 0 1.0
21619 1 1.0
21620 0 0.0
21621 0 0.0
21622 0 0.0
21623 0 0.0
21624 0 0.0
21625 0 0.0
21626 0 0.0
21627 0 0.0
21628 0 0.0
21629 0 0.0
21630 0 0.0
21631 0 0.0
21632 0 0.0
21633 0 0.0
21634 0 0.0
21635 0 0.0
21636 0 0.0
21637 0 1.0
21638 0 0.0
21639 0 0.0
21640 0 0.0
21641 0 0.0
21642 0 0.0
21643 1 0.0
21644 0 0.0
21645 0 0.0
21646 1 1.0
21647 0 0.0
21648 0 0.0
21649 0 0.0
21650 0 0.0
21651 0 0.0
21652 1 1.0
21653 0 0.0
21654 0 0.0
21655 0 0.0
21656 0 0.0
21657 0 0.0
21658 0 0.0
21659 0 0.0
21660 0 0.0
21661 0 0.0
21662 0 0.0
21663 0 0.0
21664 0 0.0
21665 0 0.0
21666 0 0.0
21667 0 0.0
21668 0 0.0
21669 0 0.0
21670 0 0.0
21671 0 0.0
21672 0 0.0
21673 0 0.0
21674 0 0.0
21675 0 0.0
21676 0 1.0
21677 0 0.0
21678 0 0.0
21679 0 0.0
21680 0 0.0
21681 0 0.0
21682 0 0.0
21683 0 0.0
21684 0 0.0
2168

22505 0 0.0
22506 0 0.0
22507 0 0.0
22508 0 0.0
22509 0 0.0
22510 0 0.0
22511 0 0.0
22512 0 0.0
22513 0 0.0
22514 0 0.0
22515 0 0.0
22516 0 0.0
22517 0 0.0
22518 1 1.0
22519 0 0.0
22520 0 0.0
22521 0 0.0
22522 0 0.0
22523 0 0.0
22524 1 1.0
22525 0 0.0
22526 0 0.0
22527 0 0.0
22528 0 0.0
22529 0 0.0
22530 0 0.0
22531 0 0.0
22532 0 0.0
22533 0 0.0
22534 0 0.0
22535 0 0.0
22536 0 0.0
22537 1 1.0
22538 0 0.0
22539 1 0.0
22540 0 0.0
22541 0 0.0
22542 0 0.0
22543 0 0.0
22544 0 0.0
22545 0 0.0
22546 0 0.0
22547 0 0.0
22548 0 0.0
22549 0 0.0
22550 0 0.0
22551 0 0.0
22552 0 1.0
22553 0 0.0
22554 0 0.0
22555 0 0.0
22556 0 0.0
22557 0 0.0
22558 0 0.0
22559 0 0.0
22560 1 1.0
22561 0 0.0
22562 0 1.0
22563 0 0.0
22564 0 0.0
22565 0 0.0
22566 0 0.0
22567 0 0.0
22568 0 0.0
22569 0 0.0
22570 0 0.0
22571 0 0.0
22572 0 0.0
22573 0 0.0
22574 0 0.0
22575 0 0.0
22576 0 0.0
22577 0 0.0
22578 0 0.0
22579 0 0.0
22580 0 1.0
22581 0 0.0
22582 0 0.0
22583 0 0.0
22584 0 0.0
22585 1 1.0
22586 0 0.0
22587 0 0.0
2258

23471 0 0.0
23472 0 0.0
23473 0 0.0
23474 0 0.0
23475 0 0.0
23476 0 0.0
23477 0 0.0
23478 0 0.0
23479 0 0.0
23480 0 0.0
23481 0 0.0
23482 1 1.0
23483 0 0.0
23484 0 0.0
23485 0 0.0
23486 0 0.0
23487 0 0.0
23488 0 0.0
23489 0 0.0
23490 0 0.0
23491 0 0.0
23492 1 1.0
23493 0 0.0
23494 0 0.0
23495 0 0.0
23496 0 1.0
23497 0 0.0
23498 0 0.0
23499 0 0.0
23500 0 0.0
23501 0 0.0
23502 0 0.0
23503 0 0.0
23504 0 0.0
23505 0 0.0
23506 0 0.0
23507 0 0.0
23508 0 0.0
23509 0 0.0
23510 0 0.0
23511 0 0.0
23512 0 0.0
23513 0 0.0
23514 0 0.0
23515 0 0.0
23516 0 0.0
23517 0 0.0
23518 0 0.0
23519 0 0.0
23520 0 0.0
23521 0 0.0
23522 0 0.0
23523 0 0.0
23524 0 0.0
23525 0 0.0
23526 0 0.0
23527 0 0.0
23528 0 0.0
23529 0 0.0
23530 1 1.0
23531 0 0.0
23532 0 0.0
23533 0 0.0
23534 0 0.0
23535 0 0.0
23536 0 0.0
23537 0 0.0
23538 0 0.0
23539 0 0.0
23540 0 0.0
23541 0 0.0
23542 0 0.0
23543 0 0.0
23544 0 0.0
23545 0 0.0
23546 0 0.0
23547 0 0.0
23548 0 0.0
23549 0 0.0
23550 0 0.0
23551 0 0.0
23552 0 0.0
23553 0 0.0
2355

In [62]:
print(first, second)

535 638


# Saving model

In [264]:
torch.save(net, "net.pt")

c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# loading and checking the net

In [337]:
loaded_model = torch.load("net.pt")

In [338]:
lst = [1, 1, 0, 1, 0, 0, 0, 1]
lst = torch.Tensor(lst)
lst

tensor([1., 1., 0., 1., 0., 0., 0., 1.])

In [267]:
lst = [1, 0, 0, 0, 0, 0, 0, 2]


CONSTANT = -1.1
lst = torch.Tensor(lst)
result = loaded_model(lst)
print(torch.argmax(result))
lst = [1 if elem[1] > CONSTANT else int(torch.argmax(elem)) for elem in result]    ## -1.39; -1.89
print(lst)
print(torch.exp(result))

tensor(0)
[0]
tensor([[0.9015, 0.0985]], grad_fn=<ExpBackward>)


In [355]:
X_test_tensor = torch.tensor(X_test.values)

result = loaded_model(X_test_tensor)
print(torch.argmax(result))
# lst = [int(torch.argmax(elem)) for elem in result]
lst = [1 if elem[1] > -1.6 else int(torch.argmax(elem)) for elem in result]    ## -1.39; -1.89

print(set(lst))
print(torch.exp(result))

tensor(47313)
{0, 1}
tensor([[0.9847, 0.0153],
        [0.9885, 0.0115],
        [0.9482, 0.0518],
        ...,
        [0.9789, 0.0211],
        [0.9936, 0.0064],
        [0.9894, 0.0106]], grad_fn=<ExpBackward>)


In [356]:
from sklearn.metrics import recall_score, accuracy_score, precision_score
accuracy_score(y_test, lst), \
            recall_score(y_test, lst), \
            precision_score(y_test, lst)

(0.9672020287404903, 0.7862595419847328, 0.8127301420305103)

In [341]:
lst

[0]